In [1]:
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
from langchain_groq import ChatGroq
import os

llm = ChatGroq(
    api_key=os.getenv("GROQ_API_KEY"),
    model = 'llama-3.1-70b-versatile',
    streaming = True,
    max_retries=2
)

In [3]:

from langchain_community.document_loaders import WebBaseLoader

loader = WebBaseLoader("https://jobs.nike.com/job/R-43863?from=job%20search%20funnel")
page_data = loader.load().pop().page_content
str(page_data)

USER_AGENT environment variable not set, consider setting it to identify your requests.


'Apply for Software Engineer III, ITC\n\nSearch JobsSkip navigationSearch JobsNIKE, INC. JOBSContract JobsJoin The Talent CommunityLife @ NikeOverviewBenefitsBrandsOverviewJordanConverseTeamsOverviewAdministrative SupportAdvanced InnovationAir Manufacturing InnovationAviationCommunicationsCustomer ServiceDesignDigitalFacilitiesFinance & AccountingGovernment & Public AffairsHuman ResourcesInsights & AnalyticsLegalManufacturing & EngineeringMarketingMerchandisingPlanningPrivacyProcurementProduct Creation, Development & ManagementRetail CorporateRetail StoresSalesSocial & Community ImpactSports MarketingStrategic PlanningSupply Chain, Distribution & LogisticsSustainabilityTechnologyLocationsOverviewNike WHQNike New York HQEHQ: Hilversum, The NetherlandsELC: Laakdal, BelgiumGreater China HQDiversity, Equity & InclusionOverviewMilitary InclusionDisability InclusionIndigenous InclusionInternshipsTechnologySoftware Engineer III, ITCKarnataka, IndiaBecome a Part of the NIKE, Inc. TeamNIKE, Inc

In [4]:
from langchain_core.prompts import PromptTemplate

prompt_extract = PromptTemplate.from_template(
    """
    ### SCRAPED TEXT FROM WEBSITE
    {page_data}
    ### INSTRUCTIONS
    The scraped text is about the job description from a company's job posting.
    Your job is to extract the job posting and return them in JSON format containing the following fields:
    - role
    - experience 
    - skills
    - description
    
    Only return the valid JSON.
    ### VALID JSON(NO PREAMBLE):
    """
)

chain_extract_data = prompt_extract | llm
result = chain_extract_data.invoke(input={"page_data": page_data})
type(result.content)


str

In [5]:
result.content

'```\n{\n    "role": "Software Engineer III, ITC",\n    "experience": "5-8 years",\n    "skills": [\n        "Marketing Automations",\n        "Adobe toolset",\n        "Adobe Experience Platform (AEP)",\n        "Adobe Journey Optimizer (AJO)",\n        "Customer Journey Analytics(CJA)",\n        "Python or Java",\n        "Sql",\n        "Data Engineering",\n        "Airflow",\n        "EMR",\n        "Hive",\n        "Spark",\n        "Microservices architecture",\n        "Domain driven design",\n        "RESTful API’s",\n        "CI/CD",\n        "DevOps"\n    ],\n    "description": "NIKE is looking for a Software engineer, to be a part of Nike’s Marketing Technology organization. This position will play a critical role, in developing Nike’s Customer Data platform, which supports and promotes our products and provides a unique user experience to all our customers in 166 countries globally."\n}\n```'

In [6]:
from langchain_core.output_parsers import JsonOutputParser

json_parser = JsonOutputParser()
json_res = json_parser.parse(result.content)
json_res

{'role': 'Software Engineer III, ITC',
 'experience': '5-8 years',
 'skills': ['Marketing Automations',
  'Adobe toolset',
  'Adobe Experience Platform (AEP)',
  'Adobe Journey Optimizer (AJO)',
  'Customer Journey Analytics(CJA)',
  'Python or Java',
  'Sql',
  'Data Engineering',
  'Airflow',
  'EMR',
  'Hive',
  'Spark',
  'Microservices architecture',
  'Domain driven design',
  'RESTful API’s',
  'CI/CD',
  'DevOps'],
 'description': 'NIKE is looking for a Software engineer, to be a part of Nike’s Marketing Technology organization. This position will play a critical role, in developing Nike’s Customer Data platform, which supports and promotes our products and provides a unique user experience to all our customers in 166 countries globally.'}

In [7]:
import pandas as pd
df = pd.read_csv('../data/my_portfolio.csv')
df.sample(5)

,Techstack,Links
5,"Flutter, Firebase, GraphQL",https://example.com/flutter-portfolio
19,"DevOps, Jenkins, Docker",https://example.com/devops-portfolio
16,"Frontend, TypeScript, Angular",https://example.com/typescript-frontend-portfolio
6,"WordPress, PHP, MySQL",https://example.com/wordpress-portfolio
8,"React Native, Node.js, MongoDB",https://example.com/react-native-portfolio


In [9]:
import chromadb
import uuid

client = chromadb.PersistentClient('../vectorstore')
collection = client.get_or_create_collection(name="portfolio")

if not collection.count():
    for _, row in df.iterrows():
        collection.add(documents=row["Techstack"],
                       metadatas={"links": row["Links"]},
                       ids=[str(uuid.uuid4())])

In [11]:
job = json_res
job['skills']

['Marketing Automations',
 'Adobe toolset',
 'Adobe Experience Platform (AEP)',
 'Adobe Journey Optimizer (AJO)',
 'Customer Journey Analytics(CJA)',
 'Python or Java',
 'Sql',
 'Data Engineering',
 'Airflow',
 'EMR',
 'Hive',
 'Spark',
 'Microservices architecture',
 'Domain driven design',
 'RESTful API’s',
 'CI/CD',
 'DevOps']

In [12]:
links = collection.query(query_texts=job['skills'], n_results=2).get('metadatas', [])
links

[[{'links': 'https://example.com/ml-python-portfolio'},
  {'links': 'https://example.com/magento-portfolio'}],
 [{'links': 'https://example.com/ios-ar-portfolio'},
  {'links': 'https://example.com/magento-portfolio'}],
 [{'links': 'https://example.com/ios-ar-portfolio'},
  {'links': 'https://example.com/xamarin-portfolio'}],
 [{'links': 'https://example.com/ml-python-portfolio'},
  {'links': 'https://example.com/vue-portfolio'}],
 [{'links': 'https://example.com/flutter-portfolio'},
  {'links': 'https://example.com/kotlin-android-portfolio'}],
 [{'links': 'https://example.com/python-portfolio'},
  {'links': 'https://example.com/ml-python-portfolio'}],
 [{'links': 'https://example.com/magento-portfolio'},
  {'links': 'https://example.com/wordpress-portfolio'}],
 [{'links': 'https://example.com/ml-python-portfolio'},
  {'links': 'https://example.com/magento-portfolio'}],
 [{'links': 'https://example.com/ml-python-portfolio'},
  {'links': 'https://example.com/ios-ar-portfolio'}],
 [{'link

In [ ]:
prompt_email = PromptTemplate.from_template(
    """
    ### JOB DESCRIPTION
    {job}
    ### INSTRUCTIONS
    Your are a Business Development Manager at JP Morgan. JP Morgan is a AI, Data Science and Data Engineering Consulting company facilitating the seamless integration of AI and Data Science into businesses through the use of cutting-edge technologies. 
    Your Job is to write cold emails to the clients regarding the job description metioned above describing the capabilities of JP Morgan and how it can help the clients in their business.
    Also add the most relevant links from the portfolio to the email to showcase the capabilities of JP Morgan {link_list}.
    Remeber you are a Sujal, Business Development Manager at JP Morgan and write the email accordingly.
    Do not provide preamble.
    ### EMAIL(NO PREAMBLE):
      
    """
)

chain_write_email = prompt_email | llm
result_email = chain_write_email.invoke(input={"job": str(job), "link_list": links})


In [18]:
print(result_email.content)

Subject: Elevate Your Customer Data Platform with Data Vanguard's Expertise

Dear Hiring Manager at Nike,

I hope this email finds you well. I came across the job description for a Software Engineer III, ITC, and I was impressed by the scope of the project. As a Business Development Manager at Data Vanguard, I believe our team can help you achieve your goals and elevate your Customer Data platform to the next level.

Our company specializes in AI, Data Science, and Data Engineering Consulting, and we have extensive experience in developing and implementing cutting-edge technologies. Our expertise in Python, Java, and Data Engineering aligns perfectly with the requirements of the project. We have a proven track record of delivering scalable and secure solutions that meet the needs of global organizations.

I'd like to highlight a few relevant projects from our portfolio that demonstrate our capabilities:

* https://example.com/ml-python-portfolio (Machine Learning and Python expertise)
